In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/teco-psda-exercisesheet1-growth-2024/growth_train.csv
/kaggle/input/teco-psda-exercisesheet1-growth-2024/growth_test_sample.csv
/kaggle/input/teco-psda-exercisesheet1-growth-2024/growth_meta.pdf
/kaggle/input/teco-psda-exercisesheet1-growth-2024/growth_test.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/teco-psda-exercisesheet1-growth-2024/growth_train.csv")
train_data.head()

,Unnamed: 0,country_name,growth,oil,rgdp60,tradeshare,yearsschool,rev_coups,assasinations
0,1,Argentina,"0,617645085",0,"4462,001465","0,156622976","4,989999771","0,933333337","1,933333278"
1,4,United States,"1,712264895",0,"9895,003906","0,160815045","8,659999847",0,"0,433333337"
2,8,Peru,"0,060205959",0,"2019,000244","0,324612707","3,019999981","0,266666681","0,566666663"
3,11,Italy,"2,932981491",0,"4564,000488","0,330021679","4,559999943","0,033333335","1,200000048"
4,12,Greece,"3,224050045",0,2093,"0,337879002","4,369999886","0,166666672","0,166666672"


In [3]:
test_data = pd.read_csv("/kaggle/input/teco-psda-exercisesheet1-growth-2024/growth_test.csv")
test_data.head()

,Unnamed: 0,country_name,oil,rgdp60,tradeshare,yearsschool,rev_coups,assasinations
0,0,India,0,"765,9998169","0,140501976","1,450000048","0,13333334","0,866666675"
1,2,Japan,0,"2953,999512","0,157703221","6,710000038",0,"0,200000003"
2,3,Brazil,0,"1783,999878","0,160405084","2,890000105","0,100000001","0,100000001"
3,5,Bangladesh,0,"951,9998169","0,22145839","0,790000021","0,306481481","0,174999997"
4,6,Spain,0,"3123,001465","0,299405932","3,799999952","0,06666667","1,433333278"


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder

y = train_data["growth"].str.replace(',', '.', regex=False)
y = y.astype(float)

features = ["country_name","oil", "rgdp60","tradeshare","yearsschool","rev_coups","assasinations"]
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_encoded = encoder.fit_transform(train_data[features])
X_test_encoded = encoder.transform(test_data[features])
X = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out())
X_test = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out())


param_grid = {
    'n_estimators': [600, 700, 500],
    'max_depth': [10, 5, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [4, 6,8]
}

rf = RandomForestRegressor(random_state=2)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, verbose=0, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

print("Best parameters found:", grid_search.best_params_)
best_rf = grid_search.best_estimator_
predictions = best_rf.predict(X_test)

output = pd.DataFrame({'Id': test_data.iloc[:,0], 'Expected': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Best parameters found: {'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 2, 'n_estimators': 600}
Your submission was successfully saved!
